First, let's load the JSON file which describes the human pose task.  This is in COCO format, it is the category descriptor pulled from the annotations file.  We modify the COCO category slightly, to add a neck keypoint.  We will use this task description JSON to create a topology tensor, which is an intermediate data structure that describes the part linkages, as well as which channels in the part affinity field each linkage corresponds to.

In [1]:
import json
import trt_pose.coco
import pandas as pd

with open('human_pose.json', 'r') as f:
    human_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(human_pose)

Next, we'll load our model.  Each model takes at least two parameters, *cmap_channels* and *paf_channels* corresponding to the number of heatmap channels
and part affinity field channels.  The number of part affinity field channels is 2x the number of links, because each link has a channel corresponding to the
x and y direction of the vector field for each link.

In [2]:
import trt_pose.models

num_parts = len(human_pose['keypoints'])
num_links = len(human_pose['skeleton'])

model = trt_pose.models.resnet18_baseline_att(num_parts, 2 * num_links).cuda().eval()

Next, let's load the model weights.  You will need to download these according to the table in the README.

In [3]:
import torch

MODEL_WEIGHTS = 'resnet18_baseline_att_224x224_A_epoch_249.pth'

model.load_state_dict(torch.load(MODEL_WEIGHTS))

<All keys matched successfully>

In order to optimize with TensorRT using the python library *torch2trt* we'll also need to create some example data.  The dimensions
of this data should match the dimensions that the network was trained with.  Since we're using the resnet18 variant that was trained on
an input resolution of 224x224, we set the width and height to these dimensions.

In [4]:
WIDTH = 224
HEIGHT = 224

data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()

Next, we'll use [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) to optimize the model.  We'll enable fp16_mode to allow optimizations to use reduced half precision.

In [5]:
import torch2trt

model_trt = torch2trt.torch2trt(model, [data], fp16_mode=True, max_workspace_size=1<<25)

The optimized model may be saved so that we do not need to perform optimization again, we can just load the model.  Please note that TensorRT has device specific optimizations, so you can only use an optimized model on similar platforms.

In [6]:
OPTIMIZED_MODEL = 'resnet18_baseline_att_224x224_A_epoch_249_trt.pth'

torch.save(model_trt.state_dict(), OPTIMIZED_MODEL)

We could then load the saved model using *torch2trt* as follows.

In [7]:
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))

<All keys matched successfully>

We can benchmark the model in FPS with the following code

In [8]:
#import time

#t0 = time.time()
#torch.cuda.current_stream().synchronize()
#for i in range(50):
#    y = model_trtmo(data)
#torch.cuda.current_stream().synchronize()
#t1 = time.time()

#print(50.0 / (t1 - t0))

Next, let's define a function that will preprocess the image, which is originally in BGR8 / HWC format.

In [9]:
import cv2
import torchvision.transforms as transforms
import PIL.Image

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
device = torch.device('cuda')

def preprocess(image):
    global device
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Next, we'll define two callable classes that will be used to parse the objects from the neural network, as well as draw the parsed objects on an image.

In [10]:
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects

parse_objects = ParseObjects(topology)
draw_objects = DrawObjects(topology)

Assuming you're using NVIDIA Jetson, you can use the [jetcam](https://github.com/NVIDIA-AI-IOT/jetcam) package to create an easy to use camera that will produce images in BGR8/HWC format.

If you're not on Jetson, you may need to adapt the code below.

In [11]:
#from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

#camera = USBCamera(width=WIDTH, height=HEIGHT, capture_fps=30)
camera = CSICamera(width=224, height=224, capture_fps=30)

camera.running = True

Next, we'll create a widget which will be used to display the camera feed with visualizations.

In [12]:
import ipywidgets
from IPython.display import display

image_w = ipywidgets.Image(format='jpeg')

display(image_w)

Image(value=b'', format='jpeg')

Finally, we'll define the main execution loop.  This will perform the following steps

1.  Preprocess the camera image
2.  Execute the neural network
3.  Parse the objects from the neural network output
4.  Draw the objects onto the camera image
5.  Convert the image to JPEG format and stream to the display widget

In [13]:
def execute(change):
    image = change['new']
    data = preprocess(image)
    cmap, paf = model_trt(data)
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = parse_objects(cmap, paf)#, cmap_threshold=0.15, link_threshold=0.15)
    draw_objects(image, counts, objects, peaks)
    image_w.value = bgr8_to_jpeg(image[:, ::-1, :])
    return peaks,image

If we call the cell below it will execute the function once on the current camera frame.

In [14]:
[peak,image]=execute({'new': camera.value})

In [15]:
import os.path
from os import path
import pandas as pd
df_name = 'ai_coach_data_1.csv'
df_name_start = 'ai_coach_data_'
df_name_end = '.csv'
while path.exists(df_name):
  file_num = int(df_name[14:-4])
  new_file_num = file_num + 1
  df_name = df_name_start+str(new_file_num)+df_name_end

In [16]:
print(df_name)

ai_coach_data_4.csv


In [41]:
#Classifier
import RPi.GPIO as GPIO
import time
from datetime import datetime
GPIO.setmode(GPIO.BOARD)
outputPin=12
Start=22
pushbutton=40
GPIO.setup(outputPin,GPIO.OUT)
GPIO.setup(Start,GPIO.OUT)
GPIO.setup(pushbutton,GPIO.IN)
GPIO.output(outputPin,1)
time.sleep(5)
GPIO.output(outputPin,0)
GPIO.output(Start,1)
time.sleep(5)
import numpy as np
counter=0
k=0
TH=0.17
Data=np.zeros((10000,37))
status=[]
timestamp=[]
display(image_w)
i=0
while (1==1):
    x=GPIO.input(pushbutton)
    if(x==0):
        GPIO.output(Start,0)
        time.sleep(0.5)
        GPIO.output(Start,1)
        peak=execute({'new': camera.value})
        print(i)
        for i in range (18):
            Data[counter][k:k+2]=peak[0][0][i][0]
            k=k+2

        if (Data[counter][8]==0 and Data[counter][12]==0) or (Data[counter][6]==0 and Data[counter][10]==0):
            print("Not detected")
            status.append("Not detected")
        elif (Data[counter][24]==0 and Data[counter][22]==0):
            if (Data[counter][10]<=0.15 or Data[counter][12]<=0.15):
                status.append("Standing position new")
                print("Standing position new")
            else:
                status.append("Seated1")
                print("Seated")
        elif (Data[counter][24]!=0 and Data[counter][22]==0):
            if Data[counter][28]!=0:
                if (Data[counter][28]-Data[counter][24])>=TH:
                    print("Standing position1")
                    status.append("Standing position1")
                else:
                    if ((Data[counter][28]-Data[counter][24])>0.65*(Data[counter][24]-Data[counter][12])):
                        print("Standing position2")
                        status.append("Standing position2")
                    else:
                        print("seated2")
                        status.append("seated2")
                    
            else:
                if (Data[counter][10]<=0.20 or Data[counter][12]<=0.2):
                    status.append("Standing position new_1")
                    print("Standing position new_1")
                else:
                    print("seated3")
                    status.append("seated3")
        elif (Data[counter][24]==0 and Data[counter][22]!=0):
            if Data[counter][26]!=0:
                if (Data[counter][26]-Data[counter][22])>=TH:
                    print("Standing position3")
                    status.append("Standing position3")
                else:
                    if((Data[counter][26]-Data[counter][22])>0.65*(Data[counter][22]-Data[counter][10])):
                        print("Standing position4")
                        status.append("Standing position4")
                    else:
                        print("seated4")
                        status.append("seated4")
   
            else:
                    if (Data[counter][10]<=0.20 or Data[counter][12]<=0.2):
                        status.append("Standing position new_2")
                        print("Standing position new_2")
                    else:
                        print("seated5") 
                        status.append("seated5")
        elif (Data[counter][24]!=0 and Data[counter][22]!=0):
            if Data[counter][28]!=0:
                if (Data[counter][28]-Data[counter][24]>=TH) or (Data[counter][26]-Data[counter][22]>=TH):
                    print("Standing position5")
                    status.append("Standing position5")
                else:
                    if ((Data[counter][26]-Data[counter][22])>0.65*(Data[counter][22]-Data[counter][10])) or ((Data[counter][28]-Data[counter][24])>0.65*(Data[counter][24]-Data[counter][12])):
                        print("Standing position6")
                        status.append("Standing position6")
                    else:    
                        print("seated6")
                        status.append("seated6")
            else:
                if (Data[counter][10]<=0.20 or Data[counter][12]<=0.2):
                    status.append("Standing position new_3")
                    print("Standing position new_3")
                else:
                    print("seated7")
                    status.append("seated7")
        counter=counter+1
        current_time=datetime.now()
        timestamp.append(current_time)
        d={'col1':status,'col2':timestamp}
        df = pd.DataFrame(data=d)
        df.to_csv(df_name)
        k=0
        time.sleep(2)
        # saving images
        if (counter%900==1):
            Image_name=current_time.strftime("%m_%d_%Y,%H:%M:%S")
            #Image_sufix=".png"
            #Image_name+=Image_sufix
            from PIL import Image as im
            data=im.fromarray(image)
            data.save('Image_'+Image_name+'.png')
        camera.unobserve_all()
    elif(x==1):
        GPIO.output(Start,0)
        camera.unobserve_all()
    #print(Data)
    i=i+1
GPIO.output(Start,0)
GPIO.cleanup()    
    

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

0
Seated


FileNotFoundError: [Errno 2] No such file or directory: 'Image_04/09/2021,13:57:52.png'

Call the cell below to attach the execution function to the camera's internal value.  This will cause the execute function to be called whenever a new camera frame is received.

In [ ]:
camera.observe(execute, names='value')

Call the cell below to unattach the camera frame callbacks.

In [ ]:
camera.unobserve_all()

In [ ]:
Data[12]

In [ ]:
from datetime import datetime
current_time=datetime.now()
df = pd.DataFrame(status)
df.insert(0,'TimeStamp',pd.to_datetime('now').replace(microsecond=0))

In [39]:
type (Image_name)

str

In [37]:
data.save('Image_name')

ValueError: unknown file extension: 

In [ ]:
from PIL import Image as im
data=im.fromarray(image)
data.save('test1.png')

In [ ]:
peak[0]

1